In [1]:
!/opt/bin/nvidia-smi
!uname -a
!python --version
# !kill -9 -1

Thu May 24 14:40:13 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.111                Driver Version: 384.111                   |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8    27W / 149W |      1MiB / 11439MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
!pip3 install torch torchvision

In [0]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision as tv
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch import optim
from pathlib import Path

%matplotlib inline

In [0]:
data_path = os.path.join(str(Path.home()), 'dev', 'pytorch-data', 'mnist')
train = tv.datasets.MNIST(data_path,
                          train=True,
                          download=True,
                          transform=transforms.ToTensor())
train_set = torch.utils.data.DataLoader(train, batch_size=128, shuffle=True)

test = tv.datasets.MNIST(data_path,
                          train=False,
                          download=True,
                          transform=transforms.ToTensor())
test_set = torch.utils.data.DataLoader(test, batch_size=128, shuffle=True)

In [5]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.conv2 = nn.Conv2d(32, 64, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(64 * 5 * 5, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)
params = list()
for n, p in net.named_parameters():
    print(n, p.size())

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=1600, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)
conv1.weight torch.Size([32, 1, 3, 3])
conv1.bias torch.Size([32])
conv2.weight torch.Size([64, 32, 3, 3])
conv2.bias torch.Size([64])
fc1.weight torch.Size([500, 1600])
fc1.bias torch.Size([500])
fc2.weight torch.Size([10, 500])
fc2.bias torch.Size([10])


In [6]:
device = torch.device(0)
print(torch.cuda.device_count())

1


In [0]:
def eval(net):
  correct, total = 0, 0
  with torch.no_grad():
      for inputs, labels in test_set:
          inputs = inputs.to(device)
          labels = labels.to(device)
          logits = net(inputs)
          _, pred = torch.max(logits, 1)
          total += len(labels)
          correct += torch.sum(labels == pred).item()
      print(correct, total)
  print("Accuracy: ", correct / total)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=1e-3, momentum=0.9)
net.to(device)
for epoch in range(300):  
    net.train()
    running_loss, num = 0.0, 0
    for i, data in enumerate(train_set, 0):
        
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()   
        
        optimizer.step()
        
        running_loss += loss.item()
        num += inputs.size(0)

    print('Epoch %d Loss: %f' % (epoch + 1, running_loss / num))
    net.eval()
    eval(net)
print('Finished Training')

Epoch 1 Loss: 0.000058
9926 10000
Accuracy:  0.9926
Epoch 2 Loss: 0.000050
9924 10000
Accuracy:  0.9924
Epoch 3 Loss: 0.000049
9923 10000
Accuracy:  0.9923
Epoch 4 Loss: 0.000042
9922 10000
Accuracy:  0.9922
Epoch 5 Loss: 0.000043
9928 10000
Accuracy:  0.9928
Epoch 6 Loss: 0.000041
9925 10000
Accuracy:  0.9925
Epoch 7 Loss: 0.000041
9925 10000
Accuracy:  0.9925
Epoch 8 Loss: 0.000039
9924 10000
Accuracy:  0.9924
Epoch 9 Loss: 0.000037
9923 10000
Accuracy:  0.9923
Epoch 10 Loss: 0.000035
9926 10000
Accuracy:  0.9926
Epoch 11 Loss: 0.000040
9923 10000
Accuracy:  0.9923
Epoch 12 Loss: 0.000040
9922 10000
Accuracy:  0.9922
Epoch 13 Loss: 0.000036
9919 10000
Accuracy:  0.9919
Epoch 14 Loss: 0.000036
9921 10000
Accuracy:  0.9921
Epoch 15 Loss: 0.000034
9920 10000
Accuracy:  0.992
Epoch 16 Loss: 0.000033
9923 10000
Accuracy:  0.9923
Epoch 17 Loss: 0.000033
9925 10000
Accuracy:  0.9925
Epoch 18 Loss: 0.000033
9923 10000
Accuracy:  0.9923
Epoch 19 Loss: 0.000030
9926 10000
Accuracy:  0.9926
Epo

Epoch 44 Loss: 0.000023
9922 10000
Accuracy:  0.9922
Epoch 45 Loss: 0.000024
9927 10000
Accuracy:  0.9927
Epoch 46 Loss: 0.000027
9925 10000
Accuracy:  0.9925
Epoch 47 Loss: 0.000022
9927 10000
Accuracy:  0.9927
Epoch 48 Loss: 0.000025
9929 10000
Accuracy:  0.9929
Epoch 49 Loss: 0.000025
9925 10000
Accuracy:  0.9925
Epoch 50 Loss: 0.000023
9928 10000
Accuracy:  0.9928
Epoch 51 Loss: 0.000024
9925 10000
Accuracy:  0.9925
Epoch 52 Loss: 0.000025
9923 10000
Accuracy:  0.9923
Epoch 53 Loss: 0.000024
9921 10000
Accuracy:  0.9921
Epoch 54 Loss: 0.000024
9922 10000
Accuracy:  0.9922
Epoch 55 Loss: 0.000021
9924 10000
Accuracy:  0.9924
Epoch 56 Loss: 0.000025
9928 10000
Accuracy:  0.9928
Epoch 57 Loss: 0.000023
9924 10000
Accuracy:  0.9924
Epoch 58 Loss: 0.000023
9927 10000
Accuracy:  0.9927
Epoch 59 Loss: 0.000024
9927 10000
Accuracy:  0.9927
Epoch 60 Loss: 0.000024
9923 10000
Accuracy:  0.9923
Epoch 61 Loss: 0.000023
9925 10000
Accuracy:  0.9925
Epoch 62 Loss: 0.000022
9925 10000
Accuracy:  

Epoch 87 Loss: 0.000020
9925 10000
Accuracy:  0.9925
Epoch 88 Loss: 0.000018
9929 10000
Accuracy:  0.9929
Epoch 89 Loss: 0.000017
9928 10000
Accuracy:  0.9928
Epoch 90 Loss: 0.000018
9927 10000
Accuracy:  0.9927
Epoch 91 Loss: 0.000019
9927 10000
Accuracy:  0.9927
Epoch 92 Loss: 0.000019
9925 10000
Accuracy:  0.9925
Epoch 93 Loss: 0.000017
9927 10000
Accuracy:  0.9927
Epoch 94 Loss: 0.000018
9926 10000
Accuracy:  0.9926
Epoch 95 Loss: 0.000019
9929 10000
Accuracy:  0.9929
Epoch 96 Loss: 0.000017
9923 10000
Accuracy:  0.9923
Epoch 97 Loss: 0.000016
9925 10000
Accuracy:  0.9925
Epoch 98 Loss: 0.000017
9928 10000
Accuracy:  0.9928
Epoch 99 Loss: 0.000019
9925 10000
Accuracy:  0.9925
Epoch 100 Loss: 0.000017
9922 10000
Accuracy:  0.9922
Epoch 101 Loss: 0.000019
9922 10000
Accuracy:  0.9922
Epoch 102 Loss: 0.000017
9925 10000
Accuracy:  0.9925
Epoch 103 Loss: 0.000018
9924 10000
Accuracy:  0.9924
Epoch 104 Loss: 0.000018
9928 10000
Accuracy:  0.9928
Epoch 105 Loss: 0.000016
9925 10000
Accur

Epoch 130 Loss: 0.000014
9932 10000
Accuracy:  0.9932
Epoch 131 Loss: 0.000016
9929 10000
Accuracy:  0.9929
Epoch 132 Loss: 0.000014
9929 10000
Accuracy:  0.9929
Epoch 133 Loss: 0.000013
9930 10000
Accuracy:  0.993
Epoch 134 Loss: 0.000013
9928 10000
Accuracy:  0.9928
Epoch 135 Loss: 0.000017
9932 10000
Accuracy:  0.9932
Epoch 136 Loss: 0.000014
9932 10000
Accuracy:  0.9932
Epoch 137 Loss: 0.000015
9929 10000
Accuracy:  0.9929
Epoch 138 Loss: 0.000015
9928 10000
Accuracy:  0.9928
Epoch 139 Loss: 0.000014
9931 10000
Accuracy:  0.9931
Epoch 140 Loss: 0.000015
9928 10000
Accuracy:  0.9928
Epoch 141 Loss: 0.000014
9928 10000
Accuracy:  0.9928
Epoch 142 Loss: 0.000014
9928 10000
Accuracy:  0.9928
Epoch 143 Loss: 0.000013
9931 10000
Accuracy:  0.9931
Epoch 144 Loss: 0.000015
9928 10000
Accuracy:  0.9928
Epoch 145 Loss: 0.000015
9928 10000
Accuracy:  0.9928
